# Steam Trike Calculations
Like many design tasks, sizing the various components of the steam trike is an iterative process. We start with certain assumed parameters and goals, then develop cylinder and accumulator dimensions. These, in turn, affect piping and generator design. At various points in the design, we can compare the results against the initial assumptions and adjust.

In this notebook, we start with one set of assumed parameters as a demonstration, but develop the code in a way that we can evaluate different sets of parameters easily.

In [1]:
# initial setup
%pip install pint
%pip install pyXSteam
import numpy as np
import pandas as pd
import pint                                 # units library
from pyntXSteam import pyntXSteam           # pint wrapper for steam library pyXSteam
ureg = pint.UnitRegistry()                  # initialize pint units registry for this notebook
ureg.autoconvert_offset_to_baseunit = True  # disable relative temperature units
ureg.default_format = "~P"                  # enable pretty unit outputs
steam = pyntXSteam(ureg)                    # initialize steam library with pint registry
x = {}                                      # iteration parameters

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Initial Parameters

In [2]:
x['trike_weight'] = 2000 * ureg('lb')        # Based on homebuilt electric and CE trikes coming in at 800-1500 lb online
x['avg_pressure'] = 125 * ureg('lbf/in**2')  # Based on max of 150 psi saturated steam with margin
x['max_velocity'] = 55 * ureg('mi/hr')       # Based on a minimum speed for highway but not freeway travel

What's the maximum RPM we can expect to be able to achieve with the reciprocating engine? Common practice says 250-500 RPM, but we can check top RPMs of prototypical locomotives and steam cars.

| Prototype | Max Speed (mi/hr) | Driver Diameter (in) | rev/min |
| --- | --- | --- | --- |
| UP 844 | 120 | 80 | 504 |
| UP 4014 | 80 | 68 | 395 |
| Reading 2102 | 80 | 70 | 384 |
| PM 1225 | 70 | 69 | 341 |
| N&W 611 | 110 | 70 | 528 |
| SP 4449 | 110 | 80 | 462 |
| Stanley Steam Car | 60 | (N/A) | 900 |
| Doble Steam Car | 70 | (N/A) | 900 |

In [3]:
x['max_rpm'] = 350 * ureg('1/min')           # Conservative bet for larger reciprocating steam engine

## Required Wheel Diameter
From the max RPM and max velocity,

In [4]:
def wheel_dia(y):
    value = y['max_velocity'] / y['max_rpm'] / np.pi

    value = value.to('in')
    y['wheel_dia'] = value
    return value
wheel_dia(x)

52.821251970155906 <Unit('inch')>

## Required Acceleration
Let's try two paths I have thought of to come up with a target maximum acceleration.

### Normal Acceleration Data
We want acceleration behavior compatible with traffic, so we can look at published data for conventional road vehicles. The data shown in Figures 1 and 2 are for consumer vehicles accelerating in normal traffic, not the maximum acceleration the vehicles are capable of. Thus, we want to target a value on the high end to reflect extra capacity available for "flooring it".


<div align="center">
<img src="fig5e.PNG" width="400"/><br />
Fig 1. Acceleration vs speed of various gasoline vehicles. Graph, "Scatter plots of acceleration-speed," from P.S.Bokare and A.K.Maurya, Acceleration-Deceleration Behaviour of Various Vehicle Types, page 10, figure 5(e).
</div><br />
<div align="center">
<img src="fig6e.PNG" width="400"/><br />
Fig 2. Idealized acceleration vs speed of gasoline vehicles. Graph, "Idealized plot of acceleration with speed for all vehicle types," from P.S.Bokare and A.K.Maurya, Acceleration-Deceleration Behaviour of Various Vehicle Types, page 11, figure 6(e).
</div><br />


The acceleration curves for these conventional vehicles include gear shifts and will not represent the curve for the steam trike. Instead, we are looking to have a comparable acceleration from a stop.

Candidate `max_accel` = 2 \[m/s^2\] ≅ 6.5 \[ft/s^2\]

### 0-60 MPH Time Data
0-60 MPH times for vehicles represent maximum acceleration, which is what we want. However, because car acceleration varies as the car accelerates, you're looking at an average max acceleration instead of the highest instantaneous acceleration. Data from ZeroTo60Times.com.

| Prototype | 0-60 MPH Time (s) | Acceleration (m/s^2) |
| --- | --- | --- |
| Geo Metro | 12.6 | 7.0 |
| Nissan Versa | 9.0 | 10.0 |
| Honda Fit | 7.8 | 11.3 |

### Selection
We see that the candidate max_accel from normal vehicle acceleration is lower than even the Geo Metro's 0-60 MPH, which makes sense as that was normal traffic acceleration instead of max. Given that we are looking for a usable max acceleration target, let's use that of the Geo Metro.

In [5]:
x['max_accel'] = 7.0 * ureg('ft/s**2')

## Cylinder Dimensions
### Required Force at Roadway
Required force on the road to accelerate `trike_weight` by `max_accel`

In [6]:
def road_force(y):
    value = y['trike_weight'] * y['max_accel']

    value = value.to('lbf')
    y['road_force'] = value
    return value
road_force(x)

435.13330240194153 <Unit('force_pound')>

### Required Cylinder Volume
The mean tractive force of a two cylinder engine, at 90 percent cutoff, with the cylinders 90 degrees out of phase is given by:

F = K2 * K3 * avg_pressure * cyl_volume / wheel_dia
* K2 = 1.2 - The ratio of mean torque of two identical piston-crank assemblies 90 degrees out of phase to the peak torque of one of the piston-crank assemblies (Jeffrey Hook, Fundamentals of Steam Locomotive Tractive Force, page 7)
* K3 = 0.9 - A fudge factor to account for pressure loss between the boiler and the cylinder (Hook)
* cyl_volume - The swept volume of the cylinder

We will solve for the swept volume of the cylinder,

In [7]:
def cyl_volume(y):
    value = road_force(y) * wheel_dia(y) * 1.2 * 0.9 / y['avg_pressure']

    value = value.to('in**3')
    y['cyl_volume'] = value
    return value
cyl_volume(x)

198.58422937057054 <Unit('inch ** 3')>

In [8]:
def max_hp(y):
    value = road_force(y) * wheel_dia(y) * np.pi * y['max_rpm']

    value = value.to('hp')
    y['road_force'] = value
    return value
max_hp(x)

63.819551018951444 <Unit('horsepower')>

### Required Cylinder Dimensions
We set cylinder bore as a parameter and calculate stroke,

In [9]:
x['cyl_bore'] = 5.0 * ureg('in')
def cyl_stroke(y):
    value = cyl_volume(y) / (y['cyl_bore']**2 * np.pi / 4)

    value = value.to('in')
    y['cyl_stroke'] = value
    return value
cyl_stroke(x)

10.113811751814735 <Unit('inch')>

## Steam System Sizing
### Maximum Steam Rate
Maximum steam rate occurs at max power output, so we need to define a reasonable target. One candidate for max power condition is full power at full throttle and full cutoff, but that is overly conservative. Insetad, lets define max power as the power required to go up a target slope at full speed.

In the vicininity of Austin, Ranch Road 2222 encounters a hill of 10% slope for most of a mile as shown in Figure 3. Not far from there, Figure 4 shows that Spicewood Springs has a 15% slope for ~500 ft. Many vehicles struggle to maintain speed going up that slope. Let's design for the longer slope and allow the energy in the accumulator to handle short bursts like Spicewood Springs.

<div align="center">
<img src="2222slope.PNG" width="400"/><br />
Fig 3. Google Earth Pro elevation profile for RM 2222 between Loop 360 and FM 620.
</div><br />
<div align="center">
<img src="spicewoodslope.PNG" width="400"/><br />
Fig 4. Google Earth Pro elevation profile for Spicewood Springs Road near Loop 360.
</div><br />

We neglect air resistance for this calculation, and assume the throttle is full open but cutoff has been linked up.

In [10]:
x['max_road_slope'] = 0.10
def max_steam_vol_rate(y):
    slope_rad = np.arctan(x['max_road_slope'])
    # convert trike_weight to force (assume Earth)
    slope_force = (y['trike_weight'].to('lb').magnitude * ureg('lbf')) * np.sin(slope_rad)
    # use Hook's equation again, this time to solve for volume of steam
    # required to maintain speed on the slope
    slope_vol = slope_force * wheel_dia(y) * 1.2 * 0.9 / y['avg_pressure']
    value = slope_vol * 4 * y['max_rpm']

    value = value.to('ft**3/min')
    y['max_steam_vol_rate'] = value
    return value
max_steam_vol_rate(x)

73.5827542012609 <Unit('foot ** 3 / minute')>

In [11]:

def max_steam_mass_rate(y):
    value = max_steam_vol_rate(y) / steam.vV_p(y['avg_pressure'] + 1 * ureg('atm'))

    value = value.to('lb/min')
    y['max_steam_mass_rate'] = value
    return value
max_steam_mass_rate(x)


22.80506769791864 <Unit('pound / minute')>

In [12]:
def max_steam_liquid_rate(y):
    value = max_steam_mass_rate(y) * steam.v_pt(1 * ureg('atm'), 25 * ureg('degC'))
    
    value = value.to('gal/min')
    y['mass_steam_liquid_rate'] = value
    return value
max_steam_liquid_rate(x)

2.740740364892484 <Unit('gallon / minute')>

There is an empirical relationship between available saturated water surface area, pressure, and the maximum steam release rate in an accumulator.

In [13]:

def accum_surface_area(y):
    area = max_steam_mass_rate(y).to('kg/hr').magnitude / 220 / (y['avg_pressure'].to('bar').magnitude + 1)
    value = (area * ureg('m**2'))

    value = value.to('ft**2')
    y['accum_surface_area'] = value
    return value
accum_surface_area(x)

3.1571179805315643 <Unit('foot ** 2')>

Next, we find the heat input required to boil water that water (constant pressure, as the feed pump/injector handles that).

In [14]:
def max_steam_heat_rate(y):
    init_h = steam.h_pt(y['avg_pressure'] + 1 * ureg('atm'), 70 * ureg('degF'))   # liquid phase
    final_h = steam.hV_p(y['avg_pressure'] + 1 * ureg('atm'), 1 * ureg(''))    # dry vapor phase
    value = max_steam_mass_rate(y) * (final_h - init_h)

    value = value.to('Btu/hr')
    y['max_steam_heat_rate'] = value
    return value
max_steam_heat_rate(x)

1580204.845531284 <Unit('british_thermal_unit / hour')>

## Evalution of Initial Iteration
Now that we have calculated a single design iteration, let's evaluate the results.

In [15]:
df = pd.DataFrame(x, index=[1])
df.style.format(".2f")
df

,trike_weight,avg_pressure,max_velocity,max_rpm,wheel_dia,max_accel,road_force,cyl_volume,cyl_bore,cyl_stroke,max_road_slope,max_steam_vol_rate,max_steam_mass_rate,mass_steam_liquid_rate,accum_surface_area,max_steam_heat_rate
1,2000 lb,125.0 lbf/in²,55.0 mi/h,350.0 1/min,52.821251970155906 in,7.0 ft/s²,435.13330240194153 lbf,198.58422937057054 in³,5.0 in,10.113811751814735 in,0.10,73.5827542012609 ft³/min,22.80506769791864 lb/min,2.740740364892484 gal/min,3.1571179805315643 ft²,1580204.845531284 Btu/h


* Wheel diameter - Road legal tires up to 53" are commercially available, but they are heavy, expensive, and wide
* Cylinder dimensions - Performing any machining operations on the 10 inch axis of the cylinder block is going to be difficult on any readily-available machine, but finding and working with stock that supports a 5 inch or larger bore will be difficult too
* Max Steam Heat Rate - 1.5 million BTU/hr is over twice what the largest DC burner available can put out, and that's not even including the inefficiency of heat transfer to the coil

From here, we can tweak assumptions as we like to get a design that looks more reasonable.

# Iterations

In [16]:
iters = [
    x,
    {
        "trike_weight": 1500 * ureg('lb'),
        "avg_pressure": 125 * ureg('lbf/in**2'),
        "max_velocity": 45 * ureg('mi/hr'),
        "max_rpm": 350 * ureg('1/min'),
        "max_accel": 7 * ureg('ft/s**2'),
        "cyl_bore": 4.5 * ureg('in'),
        "max_road_slope": 0.1 * ureg('')
    }
]

In [17]:
# run calculations
for each in iters:
    max_hp(each)
    cyl_stroke(each)
    max_steam_liquid_rate(each)
    accum_surface_area(each)
    max_steam_heat_rate(each)
# display
df = pd.DataFrame(iters)
df

,trike_weight,avg_pressure,max_velocity,max_rpm,wheel_dia,max_accel,road_force,cyl_volume,cyl_bore,cyl_stroke,max_road_slope,max_steam_vol_rate,max_steam_mass_rate,mass_steam_liquid_rate,accum_surface_area,max_steam_heat_rate
0,2000 lb,125.0 lbf/in²,55.0 mi/h,350.0 1/min,52.821251970155906 in,7.0 ft/s²,435.13330240194153 lbf,198.58422937057054 in³,5.0 in,10.113811751814735 in,0.10,73.5827542012609 ft³/min,22.80506769791864 lb/min,2.740740364892484 gal/min,3.1571179805315643 ft²,1580204.845531284 Btu/h
1,1500 lb,125.0 lbf/in²,45.0 mi/h,350.0 1/min,43.2173879755821 in,7.0 ft/s²,326.34997680145614 lbf,121.85850438648647 in³,4.5 in,7.661978599859648 in,0.1,45.15305371441009 ft³/min,13.994018814631891 lb/min,1.6818179511840239 gal/min,1.937322397144369 ft²,969671.1552123787 Btu/h
